<a href="https://colab.research.google.com/github/learn-investments/notebooks/blob/main/funds and taxes/evaluation of mutual funds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import statsmodels.api as sm
from scipy.stats import ttest_1samp as ttest
!pip install --upgrade yfinance
import yfinance as yf
import plotly.graph_objects as go

## Specify fund and benchmark

In [2]:
TICKER    = 'FCNTX'
BENCHMARK = 'SPY'

# Some potential benchmark ETFs: 
# 'SPY = S&P 500 ETF', 'IVE = S&P 500 Value ETF', 'IVW = S&P 500 Growth ETF', 
# 'IWB = Russell 1000 ETF', 'IWD = Russell 1000 Value ETF', 'IWF = Russell 1000 Growth ETF', 
# 'IWM = Russell 2000 ETF', 'IWN = Russell 2000 Value ETF', 'IWO = Russell 2000 Growth ETF', 
# 'IWV = Russell 3000 ETF'


## Pull data

In [3]:
# fund returns
rets = yf.download(TICKER,start='1970-01-01', progress=False)
rets = rets['Adj Close'].resample('M').last().pct_change().dropna()
rets.name = TICKER
rets.index = rets.index.to_period('M')

In [4]:
# benchmark returns
benchmark = yf.download(BENCHMARK,start='1970-01-01', progress=False)
benchmark = benchmark['Adj Close'].resample('M').last().pct_change().dropna()
benchmark.name = BENCHMARK
benchmark.index = benchmark.index.to_period('M')

In [5]:
# ff_monthly data
ff3    = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
ff5    = pdr('F-F_Research_Data_5_Factors_2x3','famafrench', start=1900)[0]/100
Mom    = pdr('F-F_Momentum_Factor','famafrench', start=1900)[0]/100
ST_Rev = pdr('F-F_ST_Reversal_Factor','famafrench', start=1900)[0]/100
LT_Rev = pdr('F-F_LT_Reversal_Factor','famafrench', start=1900)[0]/100
ff48   = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]/100

facts = pd.concat((ff5, Mom, ST_Rev, LT_Rev), axis=1).dropna()

In [6]:
# combine data
df = pd.concat((rets, benchmark, facts), axis=1).dropna()
df[TICKER + '-RF']   = df[TICKER] - df.RF
df[BENCHMARK +'-RF'] = df[BENCHMARK] - df.RF
df.head()

,FCNTX,SPY,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom,ST_Rev,LT_Rev,FCNTX-RF,SPY-RF
Date,,,,,,,,,,,,,
1993-02,0.006552,0.010668,0.0012,-0.0345,0.0642,-0.0042,0.0413,0.0022,0.0314,-0.0149,0.0421,0.004352,0.008468
1993-03,0.047788,0.022399,0.0230,0.0009,0.0118,-0.0015,0.0090,0.0025,0.0373,-0.0095,0.0188,0.045288,0.019899
1993-04,0.006757,-0.025588,-0.0305,-0.0085,0.0249,-0.0361,0.0138,0.0024,0.0038,-0.0171,0.0237,0.004357,-0.027988
1993-05,0.039597,0.026970,0.0289,0.0190,-0.0342,-0.0012,-0.0104,0.0022,0.0022,0.0149,0.0132,0.037397,0.024770
1993-06,0.000323,0.003607,0.0031,0.0013,0.0275,-0.0092,0.0121,0.0025,0.0455,0.0224,0.0172,-0.002177,0.001107


## Simple Benchmark Adjustment
- active return  = return - benchmark return
- tracking error = sd(active return)
- information ratio = mean(active return)/sd(active return) 

In [7]:
xrets = 100 * 100 * (df[TICKER] - df[BENCHMARK])    # expressed in basis points per month

# Statistics of interest
xrets_mn   = xrets.mean()
xrets_sd   = xrets.std()
xrets_info = xrets_mn/xrets_sd
xrets_t, xrets_p = ttest(xrets,0)
xrets_stderr   = xrets_mn/xrets_t

print(f'Simple excess-of-benchmark performance for {TICKER} relative to benchmark {BENCHMARK}:')
print(f'     mean excess return (bps):\t\t{xrets_mn:.2f}')
print(f'     tracking error (bps):\t\t{xrets_sd:.2f}')
print(f'     stderr(mean excess return): \t{xrets_stderr:.2f}')
print(f'     t-stat(mean excess return): \t{xrets_t:.2f}')
print(f'     p-value(mean excess return):\t{xrets_p:.2f}')
print(f'     information ratio:\t\t\t{xrets_info:.2%}')

Simple excess-of-benchmark performance for FCNTX relative to benchmark SPY:
     mean excess return (bps):		14.32
     tracking error (bps):		196.18
     stderr(mean excess return): 	10.30
     t-stat(mean excess return): 	1.39
     p-value(mean excess return):	0.17
     information ratio:			7.30%


In [8]:
# Cumulative returns
contribs = pd.DataFrame(dtype=float, index=df.index, columns=['Active', 'Benchmark'])
contribs['Benchmark'] = df[BENCHMARK]
contribs['Active'] = df[TICKER] - df[BENCHMARK]
cum = (1 + contribs).cumprod()

fig = go.Figure()
trace= go.Scatter(x=cum.index.astype(str), y=cum.Active, name='Active', hovertemplate="Active: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
trace= go.Scatter(x=cum.index.astype(str), y=cum.Benchmark, name='Benchmark', hovertemplate="Benchmark: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
fig.update_yaxes(title='Compound Return')
fig.update_xaxes(title="Date")
fig.update_layout(hovermode='x unified')
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()


## Beta-adjusted benchmark
- active return  = return - (risk-free + beta * (benchmark - risk-free))
- tracking error = sd(active return)
- information ratio = mean(active return)/sd(active return) 

In [9]:
y = df[TICKER + '-RF']
X = sm.add_constant(df[BENCHMARK +'-RF'])
results = sm.OLS(y,X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               FCNTX-RF   R-squared:                       0.799
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     1432.
Date:                Tue, 20 Jun 2023   Prob (F-statistic):          1.12e-127
Time:                        21:32:12   Log-Likelihood:                 929.41
No. Observations:                 363   AIC:                            -1855.
Df Residuals:                     361   BIC:                            -1847.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0024      0.001      2.369      0.0

In [10]:
# Statistics of interest
xrets_mn = 100*100*results.params['const']
xrets_sd = 100*100*np.sqrt(results.mse_resid)
xrets_t  = results.tvalues['const']
xrets_p  = results.pvalues['const']
xrets_stderr = 100*100*results.bse['const']
xrets_info   =  xrets_mn/xrets_sd
beta = results.params[BENCHMARK +'-RF']
beta_se = results.bse[BENCHMARK +'-RF']

print(f'Beta-adjusted benchmark performance for {TICKER} relative to benchmark {BENCHMARK}:')
print(f'     alpha (bps):\t{xrets_mn:.2f}')
print(f'     stderr(alpha): \t{xrets_stderr:.2f}')
print(f'     t-stat(alpha): \t{xrets_t:.2f}')
print(f'     p-value(alpha):\t{xrets_p:.2f}')
print(f'     beta:\t\t{beta:.2f}')
print(f'     stderr(beta):\t{beta_se:.2f}')
print(f'     information ratio:\t{xrets_info:.2%}')

Beta-adjusted benchmark performance for FCNTX relative to benchmark SPY:
     alpha (bps):	23.61
     stderr(alpha): 	9.97
     t-stat(alpha): 	2.37
     p-value(alpha):	0.02
     beta:		0.86
     stderr(beta):	0.02
     information ratio:	12.59%


In [11]:
# Cumulative returns
contribs = pd.DataFrame(dtype=float, index=df.index, columns=['Active', 'Benchmark'])
contribs['Benchmark'] = beta*df[BENCHMARK] + (1-beta)*df.RF
contribs['Active'] = df[TICKER] - contribs['Benchmark']
cum = (1 + contribs).cumprod()

fig = go.Figure()
trace= go.Scatter(x=cum.index.astype(str), y=cum.Active, name='Active', hovertemplate="Active: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
trace= go.Scatter(x=cum.index.astype(str), y=cum.Benchmark, name='Benchmark', hovertemplate="Benchmark: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
fig.update_yaxes(title='Compound Return')
fig.update_xaxes(title="Date")
fig.update_layout(hovermode='x unified')
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()